In [1]:
%%capture

from ablator import ModelConfig, OptimizerConfig, TrainConfig, RunConfig, ParallelConfig
from ablator import ModelWrapper, ProtoTrainer, Stateless, Derived, ParallelTrainer
from ablator.main.configs import SearchSpace

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

from sklearn.metrics import f1_score, accuracy_score

import os
import shutil

In [2]:
from ablator.main.configs import ParallelConfig, SearchSpace
search_space = {
    "train_config.optimizer_config.arguments.lr": SearchSpace(
        value_range=[0.05, 0.1], value_type="float"
    ),
    "model_config.hidden_size": SearchSpace(
        value_range=[250, 500], value_type="int"
    ),
}


class CustomModelConfig(ModelConfig):
    input_size :int
    hidden_size :int
    num_classes :int 


model_config = CustomModelConfig(input_size = 28*28, hidden_size = 256, num_classes = 10)

optimizer_config = OptimizerConfig(
    name="adam", 
    arguments={"lr": 0.001}
)

train_config = TrainConfig(
    dataset="Fashion-mnist",
    batch_size=32,
    epochs=10,
    optimizer_config=optimizer_config,
    scheduler_config=None,
    rand_weights_init = True
)

class CustomParallelConfig(ParallelConfig):
    model_config: CustomModelConfig

parallel_config = CustomParallelConfig(
    train_config=train_config,
    model_config = model_config,
    device="cpu",
    amp=False,
    search_space=search_space,
    optim_metrics={"val_loss": "min"},
    total_trials=10,
    concurrent_trials=10,
    gpu_mb_per_experiment=100,
    cpus_per_experiment=0.5,
    random_seed = 123
)

In [4]:
%%capture

import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.ToTensor()

train_dataset = torchvision.datasets.FashionMNIST(
    root='./data',
    train=True,
    download=True,
    transform=transform
)

test_dataset = torchvision.datasets.FashionMNIST(
    root='./data',
    train=False,
    download=True,
    transform=transform
)


train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True
)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False
)


In [5]:
# Define the model
class FashionMNISTModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(FashionMNISTModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        x = x.view(x.size(0), -1)  
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

# Adding loss to the model.
class MyModel(nn.Module):
    def __init__(self, config: CustomModelConfig) -> None:
        super().__init__()
        input_size = config.input_size
        hidden_size = config.hidden_size
        num_classes = config.num_classes
        
        self.model = FashionMNISTModel(input_size, hidden_size, num_classes)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x, labels=None):
        out = self.model(x)
        loss = None

        if labels is not None:
            loss = self.loss(out, labels)

        out = out.argmax(dim=-1)

        return {"y_pred": out, "y_true": labels}, loss

In [6]:
def my_accuracy(y_true, y_pred):
    return accuracy_score(y_true.flatten(), y_pred.flatten())

def my_f1_score(y_true, y_pred):
    return f1_score(y_true.flatten(), y_pred.flatten(), average='weighted')

In [7]:
class MyModelWrapper(ModelWrapper):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def make_dataloader_train(self, run_config: ParallelConfig):
        return train_dataloader

    def make_dataloader_val(self, run_config: ParallelConfig):
        return test_dataloader

    def evaluation_functions(self):
        return {
            "accuracy": my_accuracy,
            "f1_score": my_f1_score
            }

In [8]:
!ray start --head

2023-06-13 15:02:27,992	INFO usage_lib.py:521 -- Usage stats collection is enabled. To disable this, add `--disable-usage-stats` to the command that starts the cluster, or run the following command: `ray disable-usage-stats` before starting the cluster. See https://docs.ray.io/en/master/cluster/usage-stats.html for more details.
2023-06-13 15:02:27,992	INFO scripts.py:702 -- Local node IP: 127.0.0.1


Traceback (most recent call last):
  File "C:\Users\user\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\user\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\user\Desktop\ablator\env\Scripts\ray.exe\__main__.py", line 7, in <module>
  File "C:\Users\user\Desktop\ablator\env\lib\site-packages\ray\scripts\scripts.py", line 2386, in main
    return cli()
  File "C:\Users\user\Desktop\ablator\env\lib\site-packages\click\core.py", line 1130, in __call__
    return self.main(*args, **kwargs)
  File "C:\Users\user\Desktop\ablator\env\lib\site-packages\click\core.py", line 1055, in main
    rv = self.invoke(ctx)
  File "C:\Users\user\Desktop\ablator\env\lib\site-packages\click\core.py", line 1657, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "C:\Users\user\Desktop\ablator\env\lib\site-pa

In [9]:

wrapper = MyModelWrapper(
    model_class=MyModel,
)

WORKING_DIRECTORY = os.getcwd()

ablator = ParallelTrainer(
    wrapper=wrapper,
    run_config=parallel_config,
)
metrics = ablator.launch(
    working_directory=WORKING_DIRECTORY,
  ray_head_address=None,
)

2023-06-13 15:02:29,604	INFO worker.py:1352 -- Connecting to existing Ray cluster at address: 127.0.0.1:6379...
2023-06-13 15:02:29,688	INFO worker.py:1538 -- Connected to Ray cluster.
2023-06-13 15:02:29,928	INFO packaging.py:546 -- Creating a file package for local directory 'c:\Users\user\Desktop\ablator\env\lib\site-packages\ablator'.
2023-06-13 15:02:30,059	INFO packaging.py:373 -- Pushing file package 'gcs://_ray_pkg_28867304fe7d6a3a.zip' (0.58MiB) to Ray cluster...
2023-06-13 15:02:30,070	INFO packaging.py:386 -- Successfully pushed file package 'gcs://_ray_pkg_28867304fe7d6a3a.zip'.
2023-06-13 15:02:30,422	INFO packaging.py:546 -- Creating a file package for local directory 'c:\Users\user\Desktop\ablator\docs\source\notebooks'.
2023-06-13 15:02:30,494	WARNING packaging.py:420 -- File c:\Users\user\Desktop\ablator\docs\source\notebooks\data\FashionMNIST\raw\train-images-idx3-ubyte is very large (44.86MiB). Consider adding this file to the 'excludes' list to skip uploading it: `r

2023-06-13 15:02:32: Model directory: None
2023-06-13 15:02:32: Creating new model


RuntimeError: c:\Users\user\Desktop\ablator\docs\source\notebooks\experiment_1901_6c10\1901_6c10_optuna.db exists. Please remove before starting a study.